In [ ]:
import os

import dspy
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate import Evaluate
import pandas as pd
from dotenv import load_dotenv

from dataloader import build_eval_dataset, check_if_data_folder_exits

In [ ]:
# load environment variables
load_dotenv()

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_VERSION = os.getenv("AZURE_OPENAI_VERSION")

DATA_FOLDER = "data/IR-Plag-Dataset"
check_if_data_folder_exits(DATA_FOLDER)

# Eval dataset for solution

In [ ]:
eval_df = build_eval_dataset(DATA_FOLDER)
eval_df.sample(15)

# Dataset for training

In [ ]:
df = pd.read_csv("data/train.tsv", sep="\t")
df = df.sample(frac=1, random_state=1337).reset_index(drop=True)

def create_example(row: pd.Series) -> dspy.Example:
    return dspy.Example(
        code_sample_1=row["sample_1"],
        code_sample_2=row["sample_2"],
        plagiarized="Yes" if row["plagiarized"] else "No",
        explanation=row["reason"],
    ).with_inputs("code_sample_1", "code_sample_2")


train_examples = []
for _, row in df.iterrows():
    example = create_example(row)
    train_examples.append(example)

# DSPy

our "8 steps of using DSPy" could be find in [README.md](README.md)

In [ ]:
# llm setup
lm = dspy.AzureOpenAI(
    api_base=AZURE_OPENAI_ENDPOINT,
    api_version=AZURE_OPENAI_VERSION,
    deployment_id=AZURE_OPENAI_DEPLOYMENT,
    api_key=AZURE_OPENAI_KEY,
)
dspy.settings.configure(lm=lm)

In [ ]:
class Signature(dspy.Signature):
    """Detect if two code samples are plagiarized. In plagiarized field answer only : Yes if the code samples are plagiarized, No otherwise. In explenation field add the reason why the code samples are/ are not plagiarized."""

    code_sample_1 = dspy.InputField(desc="The first code sample to compare")
    code_sample_2 = dspy.InputField(desc="The second code sample to compare")
    explanation = dspy.OutputField(
        desc="Explanation or reason why the code samples are/ are not plagiarized"
    )
    plagiarized = dspy.OutputField(
        desc="Yes/No indicating if code samples are plagiarized"
    )


class CoT(dspy.Module):
    def __init__(self) -> None:
        super().__init__()
        self.prog = dspy.ChainOfThought(Signature)

    def forward(self, code_sample_1: str, code_sample_2: str) -> Signature:
        return self.prog(code_sample_1=code_sample_1, code_sample_2=code_sample_2)


In [ ]:
def validate_answer(
    example: dspy.Example, pred: Signature, trace: object = None
) -> bool:
    score = True if pred.plagiarized.lower() == example.plagiarized.lower() else False
    # print(f"Pred: {pred.plagiarized} | Actual: {example.plagiarized} | Match: {score}")
    return score

evaluate = Evaluate(
    devset=train_examples,
    metric=validate_answer,
    num_threads=4,
    display_progress=True,
    display_table=0,
)

In [ ]:
# zero-shot evaluation on train data
evaluate(CoT())

In [9]:
config = {"max_bootstrapped_demos": 8, "max_labeled_demos": 8}

teleprompter = BootstrapFewShot(metric=validate_answer, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=train_examples)
evaluate(optimized_cot)